# Day16_2: DNN (Deep Neural Network) - 심층 신경망 (정답)

## 학습 목표

**Part 1: 기초**
1. 깊은 신경망의 장점과 문제점 이해하기
2. 가중치 초기화 (Xavier, He) 이해하기
3. Batch Normalization 이해하기
4. Dropout으로 과적합 방지하기
5. 조기 종료(Early Stopping) 구현하기

**Part 2: 심화**
1. Learning Rate Scheduler 활용하기
2. 모델 저장과 로드 (torch.save/load)
3. 체크포인트 관리하기
4. Fashion-MNIST로 DNN 실습하기

---

## 환경 설정

In [ ]:
# 필수 라이브러리 임포트
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
import torchvision
import torchvision.transforms as transforms

import numpy as np
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots

# 경고 무시
import warnings
warnings.filterwarnings('ignore')

# 랜덤 시드 고정
torch.manual_seed(42)
np.random.seed(42)

# device 설정
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"PyTorch 버전: {torch.__version__}")
print(f"사용 device: {device}")

: 

## 공통 클래스 정의

In [ ]:
# Early Stopping 클래스
class EarlyStopping:
    def __init__(self, patience=5, min_delta=0.001):
        self.patience = patience
        self.min_delta = min_delta
        self.counter = 0
        self.best_loss = None
        self.early_stop = False
    
    def __call__(self, val_loss):
        if self.best_loss is None:
            self.best_loss = val_loss
        elif val_loss > self.best_loss - self.min_delta:
            self.counter += 1
            print(f"  EarlyStopping counter: {self.counter}/{self.patience}")
            if self.counter >= self.patience:
                self.early_stop = True
        else:
            self.best_loss = val_loss
            self.counter = 0
        return self.early_stop

# Model Checkpoint 클래스
class ModelCheckpoint:
    def __init__(self, filepath='best_model.pth', verbose=True):
        self.filepath = filepath
        self.verbose = verbose
        self.best_loss = float('inf')
    
    def __call__(self, val_loss, model):
        if val_loss < self.best_loss:
            if self.verbose:
                print(f"  Validation loss improved ({self.best_loss:.4f} -> {val_loss:.4f}). Saving model...")
            self.best_loss = val_loss
            torch.save(model.state_dict(), self.filepath)
            return True
        return False

print("공통 클래스 정의 완료!")

---

## 실습 퀴즈 정답

---

### Q1. 가중치 초기화 (기본)

**문제**: `nn.Linear(100, 50)` 레이어를 생성하고 He 초기화를 적용하세요.

In [ ]:
# 정답 코드
import torch
import torch.nn as nn

# 1. Linear 레이어 생성
layer = nn.Linear(100, 50)

# 2. 기본 초기화 상태 확인
print("기본 초기화:")
print(f"  평균: {layer.weight.data.mean():.6f}")
print(f"  표준편차: {layer.weight.data.std():.6f}")

# 3. He (Kaiming) 초기화 적용
nn.init.kaiming_uniform_(layer.weight, nonlinearity='relu')

# 4. He 초기화 후 상태 확인
print("\nHe 초기화 후:")
print(f"  평균: {layer.weight.data.mean():.6f}")
print(f"  표준편차: {layer.weight.data.std():.6f}")

In [ ]:
# 테스트
assert layer.weight.shape == (50, 100), "레이어 크기가 올바르지 않습니다."
print("Q1 테스트 통과!")

**풀이 설명**

- **접근 방법**: `nn.init.kaiming_uniform_()` 함수를 사용하여 He 초기화를 적용합니다.
- **핵심 개념**: He 초기화는 ReLU 활성화 함수에 최적화되어 있으며, 가중치 분산을 적절히 유지하여 기울기 소실/폭발을 방지합니다.
- **대안**: `nn.init.kaiming_normal_()`을 사용할 수도 있습니다 (균등분포 대신 정규분포).
- **실수 주의**: `nonlinearity` 파라미터를 지정하지 않으면 기본값 'leaky_relu'가 사용됩니다.
- **실무 팁**: PyTorch의 Linear 레이어는 기본적으로 좋은 초기화를 사용하지만, ReLU 계열 활성화 함수에는 He 초기화가 더 효과적입니다.

---

### Q2. BatchNorm 적용 (기본)

**문제**: 32개의 특성을 가진 배치 데이터에 BatchNorm1d를 적용하고, 적용 전후의 평균과 표준편차를 비교하세요.

In [ ]:
# 정답 코드
import torch
import torch.nn as nn

# 1. BatchNorm1d 레이어 생성
bn = nn.BatchNorm1d(num_features=32)

# 2. 배치 데이터 생성 (batch_size=64, features=32)
x = torch.randn(64, 32) * 3 + 5  # 평균 5, 표준편차 3인 데이터

# 3. 적용 전 통계량
print("BatchNorm 적용 전:")
print(f"  평균: {x.mean():.4f}")
print(f"  표준편차: {x.std():.4f}")

# 4. BatchNorm 적용 (학습 모드)
bn.train()
x_bn = bn(x)

# 5. 적용 후 통계량
print("\nBatchNorm 적용 후:")
print(f"  평균: {x_bn.mean():.4f}")
print(f"  표준편차: {x_bn.std():.4f}")

In [ ]:
# 테스트
assert abs(x_bn.mean().item()) < 0.1, "평균이 0에 가깝지 않습니다."
assert abs(x_bn.std().item() - 1.0) < 0.2, "표준편차가 1에 가깝지 않습니다."
print("Q2 테스트 통과!")

**풀이 설명**

- **접근 방법**: `nn.BatchNorm1d()`로 배치 정규화 레이어를 생성하고 데이터를 통과시킵니다.
- **핵심 개념**: BatchNorm은 미니배치의 평균을 0, 표준편차를 1로 정규화하여 학습을 안정화합니다.
- **대안**: `nn.BatchNorm2d()`는 이미지 데이터(4D 텐서)에 사용됩니다.
- **실수 주의**: `num_features`는 입력의 특성(채널) 수와 일치해야 합니다.
- **실무 팁**: BatchNorm은 학습 모드와 평가 모드에서 다르게 동작하므로 `.train()`과 `.eval()`을 적절히 사용해야 합니다.

---

### Q3. Dropout 동작 확인 (기본)

**문제**: `p=0.5`인 Dropout 레이어를 만들고, 학습 모드와 평가 모드에서의 차이를 확인하세요.

In [ ]:
# 정답 코드
import torch
import torch.nn as nn

# 1. Dropout 레이어 생성
dropout = nn.Dropout(p=0.5)

# 2. 입력 데이터 생성
x = torch.ones(10)
print(f"원본 데이터: {x}")

# 3. 학습 모드에서 Dropout 적용
dropout.train()
print("\n학습 모드 (dropout.train()):")
for i in range(3):
    output = dropout(x)
    print(f"  시도 {i+1}: {output}")
    print(f"    -> 0의 비율: {(output == 0).sum().item() / len(output) * 100:.0f}%")

# 4. 평가 모드에서 Dropout 적용
dropout.eval()
print("\n평가 모드 (dropout.eval()):")
output_eval = dropout(x)
print(f"  출력: {output_eval}")
print(f"  -> 원본과 동일 (Dropout 비활성화)")

In [ ]:
# 테스트
dropout.eval()
test_output = dropout(x)
assert torch.equal(test_output, x), "평가 모드에서 Dropout이 적용되면 안 됩니다."
print("Q3 테스트 통과!")

**풀이 설명**

- **접근 방법**: Dropout 레이어를 생성하고 `.train()`과 `.eval()` 모드에서 각각 출력을 확인합니다.
- **핵심 개념**: 학습 모드에서는 p 확률로 뉴런을 0으로 만들고, 평가 모드에서는 아무 변화 없이 통과합니다.
- **대안**: `nn.Dropout2d()`는 2D 특성맵(이미지)에 적용됩니다.
- **실수 주의**: 모델 평가 시 `.eval()` 호출을 잊으면 Dropout이 적용되어 성능이 저하됩니다.
- **실무 팁**: 학습 시 Dropout으로 비활성화된 뉴런의 출력은 1/(1-p)로 스케일링되어 평가 시와 기대값이 동일합니다.

---

### Q4. Early Stopping 구현 (응용)

**문제**: patience=3인 Early Stopping 클래스를 사용하여, 아래 검증 손실 시퀀스에서 언제 학습이 중단되는지 확인하세요.

In [ ]:
# 정답 코드

# 1. Early Stopping 인스턴스 생성
early_stopper = EarlyStopping(patience=3, min_delta=0.001)

# 2. 검증 손실 시퀀스
val_losses = [0.8, 0.6, 0.5, 0.48, 0.49, 0.50, 0.51]

# 3. 각 에포크에서 Early Stopping 체크
print("Early Stopping 시뮬레이션 (patience=3):")
print("=" * 50)

stop_epoch = None
for epoch, loss in enumerate(val_losses):
    print(f"Epoch {epoch}: val_loss = {loss}")
    
    if early_stopper(loss):
        stop_epoch = epoch
        print(f"\nEarly stopping triggered at epoch {epoch}!")
        break

if stop_epoch is None:
    print("\nEarly stopping이 발동되지 않았습니다.")

print(f"\n결과: 에포크 {stop_epoch}에서 학습 중단 (손실 개선 없이 3 에포크 경과)")

In [ ]:
# 테스트
assert stop_epoch == 6, f"에포크 6에서 중단되어야 합니다. 실제: {stop_epoch}"
print("Q4 테스트 통과!")

**풀이 설명**

- **접근 방법**: 각 에포크에서 검증 손실을 Early Stopping 객체에 전달하고, True가 반환되면 학습을 중단합니다.
- **핵심 개념**: 에포크 3(0.48)이 최저점이고, 이후 0.49, 0.50, 0.51로 3번 연속 개선되지 않아 에포크 6에서 중단됩니다.
- **대안**: Keras의 `EarlyStopping` 콜백을 참고하여 구현할 수 있습니다.
- **실수 주의**: `min_delta`가 너무 크면 작은 개선도 무시되어 조기에 중단될 수 있습니다.
- **실무 팁**: patience는 데이터셋과 모델에 따라 5~10 정도가 일반적입니다.

---

### Q5. Learning Rate Scheduler (응용)

**문제**: StepLR 스케줄러를 사용하여 10 에포크마다 학습률을 0.1배로 감소시키세요. 30 에포크 동안의 학습률 변화를 출력하세요.

In [ ]:
# 정답 코드
import torch
import torch.nn as nn
import torch.optim as optim

# 1. 간단한 모델과 옵티마이저 생성
model = nn.Linear(10, 1)
optimizer = optim.SGD(model.parameters(), lr=0.1)

# 2. StepLR 스케줄러 생성
scheduler = optim.lr_scheduler.StepLR(
    optimizer,
    step_size=10,  # 10 에포크마다
    gamma=0.1      # 학습률 * 0.1
)

# 3. 30 에포크 동안 학습률 변화 기록
lrs = []
epochs = 30

print("StepLR 스케줄러 학습률 변화:")
print("=" * 40)

for epoch in range(epochs):
    current_lr = optimizer.param_groups[0]['lr']
    lrs.append(current_lr)
    
    if epoch % 10 == 0 or epoch == epochs - 1:
        print(f"Epoch {epoch:2d}: LR = {current_lr:.6f}")
    
    # 학습 스텝 (생략)
    # ...
    
    # 스케줄러 업데이트
    scheduler.step()

In [ ]:
# 시각화
import plotly.express as px

fig = px.line(x=list(range(epochs)), y=lrs, markers=True,
              title='StepLR: 10 에포크마다 학습률 0.1배 감소',
              labels={'x': 'Epoch', 'y': 'Learning Rate'})
fig.update_yaxes(type='log')
fig.show()

In [ ]:
# 테스트
assert abs(lrs[0] - 0.1) < 1e-6, "초기 학습률이 0.1이어야 합니다."
assert abs(lrs[10] - 0.01) < 1e-6, "에포크 10에서 학습률이 0.01이어야 합니다."
assert abs(lrs[20] - 0.001) < 1e-6, "에포크 20에서 학습률이 0.001이어야 합니다."
print("Q5 테스트 통과!")

**풀이 설명**

- **접근 방법**: `StepLR` 스케줄러를 생성하고 매 에포크마다 `scheduler.step()`을 호출합니다.
- **핵심 개념**: `step_size` 에포크마다 학습률이 `gamma`배로 감소합니다.
- **대안**: `ExponentialLR`, `CosineAnnealingLR`, `ReduceLROnPlateau` 등 다양한 스케줄러를 사용할 수 있습니다.
- **실수 주의**: `scheduler.step()`을 호출하는 위치가 중요합니다 (보통 에포크 끝에서).
- **실무 팁**: `ReduceLROnPlateau`는 검증 손실이 정체될 때만 학습률을 감소시켜 더 적응적입니다.

---

### Q6. 모델 저장/로드 (응용)

**문제**: 간단한 MLP 모델을 생성하고, state_dict를 저장한 뒤 새로운 모델에 로드하세요. 두 모델의 첫 번째 레이어 가중치가 동일한지 확인하세요.

In [ ]:
# 정답 코드
import torch
import torch.nn as nn

# 1. MLP 모델 정의
class SimpleMLP(nn.Module):
    def __init__(self):
        super().__init__()
        self.fc1 = nn.Linear(10, 20)
        self.fc2 = nn.Linear(20, 5)
    
    def forward(self, x):
        x = torch.relu(self.fc1(x))
        return self.fc2(x)

# 2. 원본 모델 생성
model_original = SimpleMLP()
print("원본 모델 첫 번째 레이어 가중치 (일부):")
print(model_original.fc1.weight.data[0, :5])

# 3. state_dict 저장
torch.save(model_original.state_dict(), 'q6_model.pth')
print("\n모델 저장 완료: q6_model.pth")

# 4. 새로운 모델 생성 및 로드
model_loaded = SimpleMLP()
model_loaded.load_state_dict(torch.load('q6_model.pth'))
model_loaded.eval()

print("\n로드된 모델 첫 번째 레이어 가중치 (일부):")
print(model_loaded.fc1.weight.data[0, :5])

# 5. 동일성 확인
is_equal = torch.equal(model_original.fc1.weight.data, model_loaded.fc1.weight.data)
print(f"\n두 모델의 가중치가 동일한가? {is_equal}")

In [ ]:
# 테스트
assert is_equal, "두 모델의 가중치가 동일해야 합니다."

# 파일 정리
import os
if os.path.exists('q6_model.pth'):
    os.remove('q6_model.pth')

print("Q6 테스트 통과!")

**풀이 설명**

- **접근 방법**: `torch.save()`로 state_dict를 저장하고, `load_state_dict()`로 새 모델에 로드합니다.
- **핵심 개념**: state_dict는 모델의 학습 가능한 파라미터(가중치, 편향)를 딕셔너리 형태로 저장합니다.
- **대안**: `torch.save(model, 'model.pth')`로 전체 모델을 저장할 수 있지만, 권장되지 않습니다.
- **실수 주의**: 모델 구조가 다르면 `load_state_dict()`가 실패합니다.
- **실무 팁**: state_dict 방식은 PyTorch 버전 간 호환성이 좋고, 모델 구조를 유연하게 변경할 수 있습니다.

---

### Q7. 체크포인트 저장 (복합)

**문제**: 모델, 옵티마이저, 에포크, 손실을 포함한 체크포인트를 저장하고 로드하는 함수를 구현하세요.

In [ ]:
# 정답 코드
import torch
import torch.nn as nn
import torch.optim as optim

# 1. 체크포인트 저장 함수
def save_checkpoint(model, optimizer, epoch, loss, path='checkpoint.pth'):
    """
    학습 상태를 체크포인트로 저장
    """
    checkpoint = {
        'epoch': epoch,
        'model_state_dict': model.state_dict(),
        'optimizer_state_dict': optimizer.state_dict(),
        'loss': loss,
    }
    torch.save(checkpoint, path)
    print(f"체크포인트 저장: epoch={epoch}, loss={loss:.4f}")

# 2. 체크포인트 로드 함수
def load_checkpoint(model, optimizer, path='checkpoint.pth'):
    """
    체크포인트에서 학습 상태 복원
    """
    checkpoint = torch.load(path)
    model.load_state_dict(checkpoint['model_state_dict'])
    optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
    epoch = checkpoint['epoch']
    loss = checkpoint['loss']
    print(f"체크포인트 로드: epoch={epoch}, loss={loss:.4f}")
    return epoch, loss

# 3. 테스트용 모델과 옵티마이저 생성
model = nn.Linear(10, 1)
optimizer = optim.SGD(model.parameters(), lr=0.01)

# 4. 체크포인트 저장
save_checkpoint(model, optimizer, epoch=15, loss=0.123, path='q7_checkpoint.pth')

# 5. 새로운 모델/옵티마이저 생성 후 로드
model_new = nn.Linear(10, 1)
optimizer_new = optim.SGD(model_new.parameters(), lr=0.01)

loaded_epoch, loaded_loss = load_checkpoint(model_new, optimizer_new, path='q7_checkpoint.pth')

print(f"\n복원된 에포크: {loaded_epoch}")
print(f"복원된 손실: {loaded_loss:.4f}")

In [ ]:
# 테스트
assert loaded_epoch == 15, "에포크가 올바르게 복원되어야 합니다."
assert abs(loaded_loss - 0.123) < 1e-6, "손실이 올바르게 복원되어야 합니다."

# 파일 정리
import os
if os.path.exists('q7_checkpoint.pth'):
    os.remove('q7_checkpoint.pth')

print("Q7 테스트 통과!")

**풀이 설명**

- **접근 방법**: 모델과 옵티마이저의 state_dict, 에포크, 손실을 딕셔너리에 담아 저장합니다.
- **핵심 개념**: 체크포인트를 사용하면 학습이 중단되어도 마지막 상태에서 재개할 수 있습니다.
- **대안**: 스케줄러 상태(`scheduler.state_dict()`)도 함께 저장할 수 있습니다.
- **실수 주의**: 로드 시 모델/옵티마이저 구조가 저장 시와 동일해야 합니다.
- **실무 팁**: 긴 학습(예: 100 에포크 이상)에서는 일정 간격으로 체크포인트를 저장하는 것이 안전합니다.

---

### Q8. 과적합 진단 (복합)

**문제**: 학습 손실과 검증 손실 곡선을 보고 과적합 여부를 판단하세요. 아래 데이터를 Plotly로 시각화하고, 과적합이 시작되는 에포크를 찾으세요.

In [ ]:
# 정답 코드
import plotly.graph_objects as go
import numpy as np

# 1. 손실 데이터
train_losses = [2.0, 1.5, 1.0, 0.7, 0.5, 0.3, 0.2, 0.15, 0.1, 0.08]
val_losses = [2.1, 1.6, 1.1, 0.8, 0.7, 0.75, 0.8, 0.9, 1.0, 1.1]
epochs = list(range(len(train_losses)))

# 2. 과적합 시작 지점 찾기
# 검증 손실이 최소인 지점 이후부터 과적합
min_val_idx = np.argmin(val_losses)
overfit_start = min_val_idx

print(f"검증 손실 최소 지점: 에포크 {min_val_idx} (val_loss = {val_losses[min_val_idx]})")
print(f"과적합 시작: 에포크 {overfit_start + 1}부터 검증 손실 증가")

# 3. 시각화
fig = go.Figure()

# 학습 손실
fig.add_trace(go.Scatter(x=epochs, y=train_losses, mode='lines+markers',
                         name='Train Loss', line=dict(color='blue')))

# 검증 손실
fig.add_trace(go.Scatter(x=epochs, y=val_losses, mode='lines+markers',
                         name='Val Loss', line=dict(color='red')))

# 과적합 시작 지점 표시
fig.add_vline(x=overfit_start, line_dash='dash', line_color='green',
              annotation_text=f'과적합 시작 (epoch {overfit_start})')

fig.update_layout(title='학습 곡선: 과적합 진단',
                  xaxis_title='Epoch',
                  yaxis_title='Loss')
fig.show()

In [ ]:
# 분석 결과
print("\n과적합 분석 결과:")
print("=" * 50)
print(f"1. 에포크 0~4: 학습 손실과 검증 손실 모두 감소 (정상)")
print(f"2. 에포크 4: 검증 손실 최저점 (val_loss = {val_losses[4]})")
print(f"3. 에포크 5~: 학습 손실은 계속 감소, 검증 손실 증가 (과적합!)")
print(f"\n권장: 에포크 4에서 Early Stopping 또는 모델 저장")

In [ ]:
# 테스트
assert overfit_start == 4, f"과적합 시작은 에포크 4여야 합니다. 실제: {overfit_start}"
print("Q8 테스트 통과!")

**풀이 설명**

- **접근 방법**: 검증 손실이 최소인 지점을 찾고, 그 이후부터 과적합이 시작된다고 판단합니다.
- **핵심 개념**: 학습 손실은 계속 감소하지만 검증 손실이 증가하면 과적합입니다.
- **대안**: 학습-검증 손실 차이가 커지는 시점을 기준으로 할 수도 있습니다.
- **실수 주의**: 검증 손실의 자연스러운 변동(노이즈)과 실제 과적합을 구분해야 합니다.
- **실무 팁**: 과적합 방지를 위해 Early Stopping, Dropout, 데이터 증강 등을 사용합니다.

---

### Q9. Fashion-MNIST DNN (종합)

**문제**: 다음 구조의 DNN을 정의하고 Fashion-MNIST에서 10 에포크 학습하세요.

In [ ]:
# 정답 코드
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
import torchvision
import torchvision.transforms as transforms

# 1. 데이터 로드
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5,), (0.5,))
])

train_dataset = torchvision.datasets.FashionMNIST(
    root='./data', train=True, download=True, transform=transform
)
test_dataset = torchvision.datasets.FashionMNIST(
    root='./data', train=False, download=True, transform=transform
)

train_loader = DataLoader(train_dataset, batch_size=128, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=128, shuffle=False)

print(f"학습 데이터: {len(train_dataset)}개")
print(f"테스트 데이터: {len(test_dataset)}개")

In [ ]:
# 2. DNN 모델 정의
class FashionDNN(nn.Module):
    def __init__(self):
        super().__init__()
        self.network = nn.Sequential(
            # 레이어 1: 784 -> 512
            nn.Linear(784, 512),
            nn.BatchNorm1d(512),
            nn.ReLU(),
            nn.Dropout(0.4),
            
            # 레이어 2: 512 -> 256
            nn.Linear(512, 256),
            nn.BatchNorm1d(256),
            nn.ReLU(),
            nn.Dropout(0.4),
            
            # 레이어 3: 256 -> 128
            nn.Linear(256, 128),
            nn.BatchNorm1d(128),
            nn.ReLU(),
            nn.Dropout(0.4),
            
            # 출력층: 128 -> 10
            nn.Linear(128, 10)
        )
        
        # He 초기화
        self._init_weights()
    
    def _init_weights(self):
        for m in self.modules():
            if isinstance(m, nn.Linear):
                nn.init.kaiming_uniform_(m.weight, nonlinearity='relu')
                if m.bias is not None:
                    nn.init.zeros_(m.bias)
    
    def forward(self, x):
        x = x.view(x.size(0), -1)  # Flatten
        return self.network(x)

# 모델 생성
model = FashionDNN().to(device)
print(model)

# 파라미터 수
total_params = sum(p.numel() for p in model.parameters())
print(f"\n총 파라미터: {total_params:,}")

In [ ]:
# 3. 학습 설정
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# 4. 학습 루프
epochs = 10
history = {'train_loss': [], 'train_acc': [], 'val_loss': [], 'val_acc': []}

for epoch in range(epochs):
    # 학습
    model.train()
    train_loss = 0.0
    correct = 0
    total = 0
    
    for images, labels in train_loader:
        images, labels = images.to(device), labels.to(device)
        
        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        
        train_loss += loss.item()
        _, predicted = outputs.max(1)
        total += labels.size(0)
        correct += predicted.eq(labels).sum().item()
    
    train_loss /= len(train_loader)
    train_acc = 100. * correct / total
    
    # 검증
    model.eval()
    val_loss = 0.0
    correct = 0
    total = 0
    
    with torch.no_grad():
        for images, labels in test_loader:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            loss = criterion(outputs, labels)
            
            val_loss += loss.item()
            _, predicted = outputs.max(1)
            total += labels.size(0)
            correct += predicted.eq(labels).sum().item()
    
    val_loss /= len(test_loader)
    val_acc = 100. * correct / total
    
    # 기록
    history['train_loss'].append(train_loss)
    history['train_acc'].append(train_acc)
    history['val_loss'].append(val_loss)
    history['val_acc'].append(val_acc)
    
    print(f"Epoch {epoch+1:2d}/{epochs} | Train Loss: {train_loss:.4f} | Train Acc: {train_acc:.2f}% | Val Loss: {val_loss:.4f} | Val Acc: {val_acc:.2f}%")

print(f"\n최종 테스트 정확도: {val_acc:.2f}%")

In [ ]:
# 테스트
assert history['val_acc'][-1] > 85, "테스트 정확도가 85% 이상이어야 합니다."
print("Q9 테스트 통과!")

**풀이 설명**

- **접근 방법**: 784->512->256->128->10 구조의 DNN을 정의하고, BatchNorm, ReLU, Dropout을 순서대로 적용합니다.
- **핵심 개념**: He 초기화, BatchNorm, Dropout을 조합하여 안정적이고 일반화된 학습을 달성합니다.
- **대안**: Leaky ReLU, SeLU 등 다른 활성화 함수를 사용할 수 있습니다.
- **실수 주의**: `.train()`과 `.eval()` 모드 전환을 잊지 않아야 합니다.
- **실무 팁**: Fashion-MNIST는 비교적 간단한 데이터셋이므로 DNN으로도 88-90% 정확도를 달성할 수 있습니다.

---

### Q10. 종합 파이프라인 (종합)

**문제**: Fashion-MNIST 학습 파이프라인을 완성하세요. 다음 요소를 모두 포함해야 합니다.

In [ ]:
# 정답 코드
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
import torchvision
import torchvision.transforms as transforms
import plotly.graph_objects as go
from plotly.subplots import make_subplots

# 1. 데이터 준비
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5,), (0.5,))
])

train_dataset = torchvision.datasets.FashionMNIST(
    root='./data', train=True, download=True, transform=transform
)
test_dataset = torchvision.datasets.FashionMNIST(
    root='./data', train=False, download=True, transform=transform
)

train_loader = DataLoader(train_dataset, batch_size=128, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=128, shuffle=False)

In [ ]:
# 2. DNN 모델 정의 (He 초기화, BatchNorm, Dropout)
class CompleteDNN(nn.Module):
    def __init__(self, input_dim=784, hidden_dims=[256, 128, 64], output_dim=10, dropout_rate=0.3):
        super().__init__()
        
        layers = []
        prev_dim = input_dim
        
        for hidden_dim in hidden_dims:
            layers.append(nn.Linear(prev_dim, hidden_dim))
            layers.append(nn.BatchNorm1d(hidden_dim))
            layers.append(nn.ReLU())
            layers.append(nn.Dropout(dropout_rate))
            prev_dim = hidden_dim
        
        layers.append(nn.Linear(prev_dim, output_dim))
        self.network = nn.Sequential(*layers)
        
        # He 초기화
        self._init_weights()
    
    def _init_weights(self):
        for m in self.modules():
            if isinstance(m, nn.Linear):
                nn.init.kaiming_uniform_(m.weight, nonlinearity='relu')
                if m.bias is not None:
                    nn.init.zeros_(m.bias)
    
    def forward(self, x):
        x = x.view(x.size(0), -1)
        return self.network(x)

# 모델 생성
model = CompleteDNN().to(device)
print(model)

In [ ]:
# 3. 학습 설정
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# ReduceLROnPlateau 스케줄러
scheduler = optim.lr_scheduler.ReduceLROnPlateau(
    optimizer, mode='min', patience=3, factor=0.5, verbose=True
)

# Early Stopping (patience=5)
early_stopper = EarlyStopping(patience=5)

# 최고 모델 체크포인트
model_ckpt = ModelCheckpoint('q10_best_model.pth')

print("학습 설정 완료!")

In [ ]:
# 4. 학습 함수
def train_epoch(model, train_loader, criterion, optimizer, device):
    model.train()
    running_loss = 0.0
    correct = 0
    total = 0
    
    for images, labels in train_loader:
        images, labels = images.to(device), labels.to(device)
        
        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        
        running_loss += loss.item()
        _, predicted = outputs.max(1)
        total += labels.size(0)
        correct += predicted.eq(labels).sum().item()
    
    return running_loss / len(train_loader), 100. * correct / total

def evaluate(model, test_loader, criterion, device):
    model.eval()
    running_loss = 0.0
    correct = 0
    total = 0
    
    with torch.no_grad():
        for images, labels in test_loader:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            loss = criterion(outputs, labels)
            
            running_loss += loss.item()
            _, predicted = outputs.max(1)
            total += labels.size(0)
            correct += predicted.eq(labels).sum().item()
    
    return running_loss / len(test_loader), 100. * correct / total

In [ ]:
# 5. 학습 실행
epochs = 30
history = {'train_loss': [], 'train_acc': [], 'val_loss': [], 'val_acc': [], 'lr': []}

print("학습 시작!")
print("=" * 70)

for epoch in range(epochs):
    # 학습
    train_loss, train_acc = train_epoch(model, train_loader, criterion, optimizer, device)
    
    # 검증
    val_loss, val_acc = evaluate(model, test_loader, criterion, device)
    
    # 학습률
    current_lr = optimizer.param_groups[0]['lr']
    
    # 기록
    history['train_loss'].append(train_loss)
    history['train_acc'].append(train_acc)
    history['val_loss'].append(val_loss)
    history['val_acc'].append(val_acc)
    history['lr'].append(current_lr)
    
    print(f"Epoch {epoch+1:2d}/{epochs} | Train Loss: {train_loss:.4f} | Train Acc: {train_acc:.2f}% | Val Loss: {val_loss:.4f} | Val Acc: {val_acc:.2f}% | LR: {current_lr:.6f}")
    
    # 스케줄러 업데이트
    scheduler.step(val_loss)
    
    # 체크포인트 저장
    model_ckpt(val_loss, model)
    
    # Early Stopping 체크
    if early_stopper(val_loss):
        print(f"\nEarly stopping at epoch {epoch+1}!")
        break

print("\n학습 완료!")

In [ ]:
# 6. 학습 곡선 시각화 (Plotly)
fig = make_subplots(rows=1, cols=3, subplot_titles=['Loss', 'Accuracy', 'Learning Rate'])

# Loss
fig.add_trace(go.Scatter(y=history['train_loss'], name='Train Loss', line=dict(color='blue')), row=1, col=1)
fig.add_trace(go.Scatter(y=history['val_loss'], name='Val Loss', line=dict(color='red')), row=1, col=1)

# Accuracy
fig.add_trace(go.Scatter(y=history['train_acc'], name='Train Acc', line=dict(color='blue')), row=1, col=2)
fig.add_trace(go.Scatter(y=history['val_acc'], name='Val Acc', line=dict(color='red')), row=1, col=2)

# Learning Rate
fig.add_trace(go.Scatter(y=history['lr'], name='LR', line=dict(color='green')), row=1, col=3)

fig.update_layout(title='Q10: 종합 학습 파이프라인', height=400)
fig.show()

In [ ]:
# 7. 최고 성능 모델 로드 및 최종 평가
model.load_state_dict(torch.load('q10_best_model.pth'))
model.eval()

final_loss, final_acc = evaluate(model, test_loader, criterion, device)
print(f"\n최고 성능 모델 테스트 정확도: {final_acc:.2f}%")

In [ ]:
# 테스트
assert final_acc > 85, "테스트 정확도가 85% 이상이어야 합니다."
assert len(history['train_loss']) > 0, "학습 기록이 있어야 합니다."

# 파일 정리
import os
if os.path.exists('q10_best_model.pth'):
    os.remove('q10_best_model.pth')

print("Q10 테스트 통과!")

**풀이 설명**

- **접근 방법**: DNN, Adam, ReduceLROnPlateau, Early Stopping, 체크포인트를 모두 조합하여 완전한 학습 파이프라인을 구성합니다.
- **핵심 개념**: 각 컴포넌트가 협력하여 안정적이고 효율적인 학습을 달성합니다.
- **대안**: SGD+Momentum, CosineAnnealingLR 등 다른 조합도 가능합니다.
- **실수 주의**: Early Stopping이 너무 일찍 발동되지 않도록 patience를 적절히 설정해야 합니다.
- **실무 팁**: 이 패턴은 대부분의 딥러닝 프로젝트에서 재사용할 수 있는 표준적인 구조입니다.

---

## 학습 정리

### Part 1: 기초 핵심 요약

| 개념 | 핵심 내용 | 언제 사용? |
|-----|----------|----------|
| 가중치 초기화 | Xavier(Sigmoid/Tanh), He(ReLU) | 모든 신경망 |
| BatchNorm | 미니배치 정규화 | 깊은 네트워크, 빠른 수렴 |
| Dropout | 무작위 뉴런 비활성화 | 과적합 방지 |
| Early Stopping | 검증 손실 모니터링 | 자동 학습 종료 |

### Part 2: 심화 핵심 요약

| 개념 | 핵심 메서드 | 언제 사용? |
|-----|-----------|----------|
| LR Scheduler | StepLR, ReduceLROnPlateau | 학습률 동적 조정 |
| 모델 저장 | torch.save(state_dict) | 학습 결과 보존 |
| 체크포인트 | 모델+옵티마이저+에포크 | 학습 재개, 최고 모델 저장 |

### DNN 모델 구조 패턴

```python
# 권장 패턴
Linear -> BatchNorm -> ReLU -> Dropout
Linear -> BatchNorm -> ReLU -> Dropout
...
Linear (출력층)
```

### 실무 팁

1. **He 초기화**: ReLU 계열 활성화 함수와 함께 사용
2. **BatchNorm**: 학습 속도 향상, 더 높은 학습률 사용 가능
3. **Dropout**: 0.3~0.5 범위로 시작, 필요에 따라 조정
4. **Early Stopping**: patience=5~10 권장
5. **체크포인트**: 긴 학습 시 반드시 사용
6. **eval() 모드**: 평가 시 반드시 `model.eval()` 호출